In [1]:
from rdflib import Namespace
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, TemplateLibrary

We need a namespace (`BLDG`) to name the entities that will exist in our model. `NUM_VAVs` will be used to generate a number of VAVs for this example

In [2]:
NUM_VAVs = 3
BLDG = Namespace("urn:my-building/")

Create an in-memory BuildingMOTIF instance and create a `my-building` model that we will populate by evaluating templates

In [3]:
bm = BuildingMOTIF("sqlite://")
bldg = Model.create("my-building")

Load in templates / classes from the Brick ontology (this may take 1-2 minutes):

In [4]:
TemplateLibrary.load(ontology_graph="../libraries/brick/Brick-subset.ttl")

TemplateLibrary(_id=1, _name=rdflib.term.URIRef('https://brickschema.org/schema/1.3-subset/Brick'), _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x7fca982cbf70>)

We load the G36 template library and pull out the "vav-cooling-only" template, corresponding to the pointlist in Section 4.1 of Guideline 36

In [5]:
lib = TemplateLibrary.load(directory="../libraries/ashrae/guideline36/")
vav_templ = lib.get_template_by_name("vav-cooling-only")

Now that we have the template, we do the following to create the VAV:

In [6]:
# create a "name" for the VAV
vav_name = BLDG["vav-0"]
# evaluate the template with that name to make sure the VAV has that name
tmp = vav_templ.evaluate({"name": vav_name})
# check what parameters are left
print(tmp.parameters)

{'dmp', 'ztemp', 'co2', 'dat', 'zone', 'occ'}


If we don't want to provide names for these yet, we can call `.fill()` to invent names for them. This is helpful for testing and prototyping

In [7]:
bindings, vav_graph = tmp.fill(BLDG) # tell 'fill' to put the invented names in the BLDG namespace

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/vav-0> a brick:VAV ;
    brick:feeds <urn:my-building/zone_1bc75c9f> ;
    brick:hasPart <urn:my-building/dmp_484c27c4> ;
    brick:hasPoint <urn:my-building/co2_ec47f443>,
        <urn:my-building/dat_9544cf5e>,
        <urn:my-building/occ_00aa94ec>,
        <urn:my-building/ztemp_5df32fc9> .


{'occ', 'co2'}


The `vav_graph` object can now be added to our model:

In [8]:
bldg.add_graph(vav_graph)

In [9]:
# run for the other n-1 VAVs
for vav in range(1,NUM_VAVs):
    vav_name = BLDG[f"vav-{vav}"]
    _, vav_graph = vav_templ.evaluate({"name": vav_name}).fill(BLDG)
    bldg.add_graph(vav_graph)

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/vav-1> a brick:VAV ;
    brick:feeds <urn:my-building/zone_fe7a3564> ;
    brick:hasPart <urn:my-building/dmp_d9c867cf> ;
    brick:hasPoint <urn:my-building/co2_746aed02>,
        <urn:my-building/dat_a54314f8>,
        <urn:my-building/occ_2e6a4481>,
        <urn:my-building/ztemp_e9893ebc> .


{'occ', 'co2'}
@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/vav-2> a brick:VAV ;
    brick:feeds <urn:my-building/zone_b897d0fb> ;
    brick:hasPart <urn:my-building/dmp_42ec6dbf> ;
    brick:hasPoint <urn:my-building/co2_e18be227>,
        <urn:my-building/dat_047ccbc7>,
        <urn:my-building/occ_5d3c3fe6>,
        <urn:my-building/ztemp_17cb4e3e> .


{'occ', 'co2'}


Print out the resulting model

In [10]:
print(bldg.graph.serialize(format="turtle"))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/vav-0> a brick:VAV ;
    brick:feeds <urn:my-building/zone_1bc75c9f> ;
    brick:hasPart <urn:my-building/dmp_484c27c4> ;
    brick:hasPoint <urn:my-building/co2_ec47f443>,
        <urn:my-building/dat_9544cf5e>,
        <urn:my-building/occ_00aa94ec>,
        <urn:my-building/ztemp_5df32fc9> .

<urn:my-building/vav-1> a brick:VAV ;
    brick:feeds <urn:my-building/zone_fe7a3564> ;
    brick:hasPart <urn:my-building/dmp_d9c867cf> ;
    brick:hasPoint <urn:my-building/co2_746aed02>,
        <urn:my-building/dat_a54314f8>,
        <urn:my-building/occ_2e6a4481>,
        <urn:my-building/ztemp_e9893ebc> .

<urn:my-building/vav-2> a brick:VAV ;
    brick:feeds <urn:my-building/zone_b897d0fb> ;
    brick:hasPart <urn:my-building/dmp_42ec6dbf> ;
    brick:hasPoint <urn:my-building/co2_e18be227>,
        <urn:my-building/dat_047ccbc7>,
        <urn:my-building/occ_5d3c3fe6>,
        <urn:my-building/ztemp_17cb4e3e> .



In [11]:
# and save your work!
bm.session.commit()

Rewinding a little bit, you may have noticed that the generated graph above doesn't actually contain any metadata for the points and parts associated with the VAVs. This is because we didn't deal with dependencies at all. We can use `inline_dependencies()` to resolve other definitions required by the VAV template:

In [12]:
# without resolving dependencies
_, g = vav_templ.fill(BLDG)
print(g.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/name_4fd91109> a brick:VAV ;
    brick:feeds <urn:my-building/zone_b8c7c121> ;
    brick:hasPart <urn:my-building/dmp_d9bead49> ;
    brick:hasPoint <urn:my-building/co2_875cafd1>,
        <urn:my-building/dat_630c7f67>,
        <urn:my-building/occ_6f8f8443>,
        <urn:my-building/ztemp_e6017a50> .


{'occ', 'co2'}
@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/name_4fd91109> a brick:VAV ;
    brick:feeds <urn:my-building/zone_b8c7c121> ;
    brick:hasPart <urn:my-building/dmp_d9bead49> ;
    brick:hasPoint <urn:my-building/co2_875cafd1>,
        <urn:my-building/dat_630c7f67>,
        <urn:my-building/occ_6f8f8443>,
        <urn:my-building/ztemp_e6017a50> .




In [13]:
# with resolved dependencies
inlined = vav_templ.inline_dependencies()
_, g = inlined.fill(BLDG)
print(g.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/name_8e0242c1> a brick:VAV ;
    brick:feeds <urn:my-building/zone_c2f3778a> ;
    brick:hasPart <urn:my-building/dmp_1e15ef83> ;
    brick:hasPoint <urn:my-building/co2_8d77633c>,
        <urn:my-building/dat_7e538b48>,
        <urn:my-building/occ_56861acb>,
        <urn:my-building/ztemp_ccc1bb2d> .

<urn:my-building/co2_8d77633c> a brick:CO2_Level_Sensor .

<urn:my-building/dat_7e538b48> a brick:Discharge_Air_Temperature_Sensor .

<urn:my-building/dmp_1e15ef83> a brick:Damper ;
    brick:hasPoint <urn:my-building/dmppos-dampera9a07710-inlined_3bc6c284> .

<urn:my-building/dmppos-dampera9a07710-inlined_3bc6c284> a brick:Damper_Position_Command .

<urn:my-building/occ_56861acb> a brick:Occupancy_Sensor .

<urn:my-building/zone_c2f3778a> a brick:HVAC_Zone .

<urn:my-building/ztemp_ccc1bb2d> a brick:Zone_Air_Temperature_Sensor .


{'occ', 'co2'}
@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-b

In [14]:
print(inlined.body.serialize(format='turtle'))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:___param___#name> a brick:VAV ;
    brick:feeds <urn:___param___#zone> ;
    brick:hasPart <urn:___param___#dmp> ;
    brick:hasPoint <urn:___param___#co2>,
        <urn:___param___#dat>,
        <urn:___param___#occ>,
        <urn:___param___#ztemp> .

<urn:___param___#co2> a brick:CO2_Level_Sensor .

<urn:___param___#dat> a brick:Discharge_Air_Temperature_Sensor .

<urn:___param___#dmp> a brick:Damper ;
    brick:hasPoint <urn:___param___#dmppos-dampera9a07710-inlined> .

<urn:___param___#dmppos-dampera9a07710-inlined> a brick:Damper_Position_Command .

<urn:___param___#occ> a brick:Occupancy_Sensor .

<urn:___param___#zone> a brick:HVAC_Zone .

<urn:___param___#ztemp> a brick:Zone_Air_Temperature_Sensor .




In [15]:
lib = TemplateLibrary.load(directory="../libraries/bca-chiller-plant/")

In [16]:
chiller = lib.get_template_by_name("chiller")
inlined = chiller.inline_dependencies()
_, g = inlined.fill(BLDG)
print(g.serialize())

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<urn:my-building/name_5335449a> a brick:Chiller ;
    brick:hasPart <urn:my-building/chwp_76027532>,
        <urn:my-building/cnd_862dc767>,
        <urn:my-building/cvlv_d93c220d> ;
    brick:hasPoint <urn:my-building/chwrt_ab777263>,
        <urn:my-building/chwst_e403b6e6> ;
    brick:isMeteredBy <urn:my-building/meter_b4d5d3a8> .

<urn:my-building/breaker_status-electrical_meter4c36a0ad-inlined_be6aa21f> a <breaker_status> .

<urn:my-building/breaker_status-electrical_meterba47275f-inlined_49d2f745> a <breaker_status> .

<urn:my-building/breaker_status-electrical_metercb831ba8-inlined_6938de81> a <breaker_status> .

<urn:my-building/bypass_valve-pumpa8f8275b-inlined_362c5ab9> a brick:Bypass_Valve ;
    brick:hasPoint <urn:my-building/vlv_cmd-bypass_valve3ab9f76c-inlined_eee18a54> .

<urn:my-building/bypass_valve-pumpcdfbe8a9-inlined_33c9ab27> a brick:Bypass_Valve ;
    

In [17]:
bm.session.rollback()